In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
dataset = pd.read_csv('credit_simple.csv', sep=';')
dataset.shape

In [ ]:
dataset.head()

In [ ]:
# SEPARANDO A CLASSE DO RESTO
y = dataset['CLASSE']
X = dataset.iloc[:, :-1]

In [ ]:
# LIMPEZA DE DADOS

In [ ]:
# VENDO QUANTOS DADOS NULOS EU TENHO NO 'X'
X.isnull().sum()

In [ ]:
# PREENCHENDO OS VALORES NULOS COM A MEDIANA DAQUELE ATRIBUTO
mediana = X['SALDO_ATUAL'].median()

X['SALDO_ATUAL'].fillna(mediana, inplace=True)

X.isnull().sum()

In [ ]:
# VERIFICANDO A MODA DO ATRIBUTO ESTADO CIVIL
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

In [ ]:
# ADICIONANDO A MODA AOS VALORES NULOS
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

In [ ]:
# ELIMINANDO OUTLIERS

# CALCULANDO O DESVIO PADRÃO
desv = X['SALDO_ATUAL'].std()
desv

In [ ]:
# VENDO SE EXISTE INSTÂNCIAS QUE SÃO MAIORES OU IGUAIS A 2 DESV. PAD.
X.loc[X['SALDO_ATUAL']>=2 * desv, 'SALDO_ATUAL']

In [ ]:
# ATRIBUIR A MEDIANA DO 'SALDO_ATUAL' PARA OS OUTLIERS
X.loc[X['SALDO_ATUAL']>=2 * desv, 'SALDO_ATUAL'] = mediana

In [ ]:
X.loc[X['SALDO_ATUAL']>=2 * desv]

In [ ]:
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

In [ ]:
# BINNING
X.loc[X['PROPOSITO'] == 'Eletrodomésticos', 'PROPOSITO'] = 'outros'
X.loc[X['PROPOSITO'] == 'qualificação', 'PROPOSITO'] = 'outros'
agrupado = X.groupby(['PROPOSITO']).size()
agrupado

In [ ]:
# EXTRAÇÃO DE CARACTERÍSTICAS
X['DATA']

In [ ]:
X['DATA'] = pd.to_datetime(X['DATA'], format='%d/%m/%Y')

In [ ]:
X['DATA']

In [ ]:
# CRIANDO DE FATO AS NOVAS CARACTERÍSTICAS
X['ANO'] = X['DATA'].dt.year
X['MES'] = X['DATA'].dt.month
X['DIASEMANA'] = X['DATA'].dt.day_name()

In [ ]:
X['DIASEMANA']

In [ ]:
# CODIFICANDO CATEGORIAS (LABEL ENCODER)
labelencoder1 = LabelEncoder()
X['ESTADOCIVIL'] = labelencoder1.fit_transform(X['ESTADOCIVIL'])
X['PROPOSITO'] = labelencoder1.fit_transform(X['PROPOSITO'])
X['DIASEMANA'] = labelencoder1.fit_transform(X['DIASEMANA'])

In [ ]:
X.head()

In [ ]:
outros = X['OUTROSPLANOSPGTO'].unique()
outros

In [ ]:
# ONDE-HOT-ENCODING PARA CODIFICAR AS CATEGORIAS
z = pd.get_dummies(X['OUTROSPLANOSPGTO'], prefix='OUTROS')
z

In [ ]:
# PADRONIZANDO OS VALORES NUMÉRICOS
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:, 0:3])
m

In [ ]:
# CONCATENANDO O DF 'z' E O OBJETO 'm' NO DF 'X'
X = pd.concat([X, z, pd.DataFrame(m, columns=['SALDO_ATUAL_N', 'RESIDENCIADESDE_N', 'IDADE_N'])], axis=1)

In [ ]:
X.head()

In [ ]:
# DELETANDO OS ATRIBUTOS QUE EU NÃO PRECISO MAIS -> 'inplace=True' é para modificar nos dados originais
X.drop(columns=['SALDO_ATUAL', 'RESIDENCIADESDE', 'IDADE', 'OUTROSPLANOSPGTO', 'DATA', 'OUTROS_banco'], inplace=True)

In [ ]:
X